<a href="https://colab.research.google.com/github/PsyCharan17/pytorch-adventures/blob/main/nanogpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [ ]:
with open("input.txt", "r", encoding='utf-8')as f:
  text = f.read()

In [ ]:
print("length of dataset in characters: ",len(text))

In [ ]:
print(text[:1000])

In [ ]:
#all unique characters that in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

In [ ]:
#create a mapping from characters to integers
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i ,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hii there"))
print(decode(encode("hii there")))


In [ ]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

In [ ]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]
len(train_data) , len(val_data) , len(train_data)+len(val_data)

In [ ]:
# dataloader : batches of chunks of data. Passing chunks of data to the transformer
block_size = 8
train_data[:block_size+1]

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target: {target}")

In [ ]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  print(f"ix shape {ix.shape} and ix: {ix}")
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x ,y

xb ,yb = get_batch('train')
#side quest - decoding the random text choosen
# print(xb.dim(), xb.shape)
# results = []
# for i in range(xb.shape[0]):
#     sequence = xb[i].tolist()
#     decoded = decode(sequence)
#     results.append(f"Batch {i}: '{decoded}'")
# print(results)
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b,:t+1]
    target = yb[b,t]
    print(f"when input is {context.tolist()} the target: {target}")



In [ ]:
print(xb) , print(yb)

In [ ]:
vocab_size